## Hyperband with Optuna

In this notebook, we'll implement [hyperband](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.HyperbandPruner.html) with Optuna.

In [1]:
import numpy as np

import optuna

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import xgboost as xgb

In [2]:
# load dataset and prepare data

data, target = load_breast_cancer(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.25)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

## Define the objective function

Check out [xgboost pruning integration](https://optuna-integration.readthedocs.io/en/stable/reference/generated/optuna_integration.XGBoostPruningCallback.html#optuna_integration.XGBoostPruningCallback)

Taken from https://github.com/optuna/optuna-examples/blob/main/xgboost/xgboost_integration.py

In [3]:
def objective(trial):

    # hyperparameter space
    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }

    # conditional space: some hyperparams depend on other hyperparams
    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # Add a callback for pruning.
    # This is the stopping criteria for successive halving: the accuracy after each round of training
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
    
    # set up the model
    bst = xgb.train(param, dtrain, evals=[(dtest, "validation")], callbacks=[pruning_callback])
    
    # evaluate
    preds = bst.predict(dtest)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_test, pred_labels)
    
    return accuracy

In [4]:
study = optuna.create_study(
    
    # a way to sample hyperparameters to create the configurations
    sampler=optuna.samplers.RandomSampler(),
    
    pruner=optuna.pruners.HyperbandPruner(
        
        # controls the minimum validation rounds that it needs to wait before stopping
        min_resource=1,
        
        # the maximum budget that we have,
        max_resource=81,
        
        # how many configurations to promote to the next round
        reduction_factor=3,
    ),
    
    direction="maximize",

)

study.optimize(
    objective, 
    
    # this parameter limits the number of configurations that
    # will be examined
    n_trials=50,
)

[I 2024-09-21 13:25:56,011] A new study created in memory with name: no-name-4126eb46-dae3-4fd1-b3b2-b4e6994634e1


[0]	validation-auc:0.85852
[1]	validation-auc:0.85852
[2]	validation-auc:0.85852
[3]	validation-auc:0.85852
[4]	validation-auc:0.85852
[5]	validation-auc:0.85852
[6]	validation-auc:0.85852
[7]	validation-auc:0.85852
[8]	validation-auc:0.85852
[9]	validation-auc:0.85852


[I 2024-09-21 13:25:56,078] Trial 0 finished with value: 0.6363636363636364 and parameters: {'booster': 'gbtree', 'lambda': 0.6779170258960109, 'alpha': 3.5787357094456264e-07, 'max_depth': 1, 'eta': 4.979509321138854e-08, 'gamma': 0.10938127206210227, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.6363636363636364.


[0]	validation-auc:0.97041
[1]	validation-auc:0.97041
[2]	validation-auc:0.97052
[3]	validation-auc:0.97010
[4]	validation-auc:0.96925
[5]	validation-auc:0.96925
[6]	validation-auc:0.96925
[7]	validation-auc:0.96925
[8]	validation-auc:0.96915
[9]	validation-auc:0.96946


[I 2024-09-21 13:25:56,224] Trial 1 finished with value: 0.6363636363636364 and parameters: {'booster': 'gbtree', 'lambda': 0.31562773258868637, 'alpha': 0.0003806986024456386, 'max_depth': 8, 'eta': 4.6724929262370215e-06, 'gamma': 0.0020512280775569075, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.6363636363636364.


...

[I 2024-09-21 13:25:58,129] Trial 47 pruned. Trial was pruned at iteration 3.


[0]	validation-auc:0.95287


[I 2024-09-21 13:25:58,139] Trial 48 pruned. Trial was pruned at iteration 1.


[0]	validation-auc:0.96946
[1]	validation-auc:0.96936
[2]	validation-auc:0.96904


[I 2024-09-21 13:25:58,158] Trial 49 pruned. Trial was pruned at iteration 3.


In [5]:
# the best hyperparameters

study.best_params

{'booster': 'dart',
 'lambda': 1.9477657049661825e-07,
 'alpha': 0.09713832092420198,
 'max_depth': 3,
 'eta': 0.23209755997424594,
 'gamma': 0.0001605470943614057,
 'grow_policy': 'lossguide',
 'sample_type': 'weighted',
 'normalize_type': 'tree',
 'rate_drop': 0.011846625311804734,
 'skip_drop': 1.6853266781086597e-06}

In [6]:
# the best performance value

study.best_value

0.965034965034965

In [7]:
r = study.trials_dataframe()

r

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_booster,params_eta,params_gamma,params_grow_policy,params_lambda,params_max_depth,params_normalize_type,params_rate_drop,params_sample_type,params_skip_drop,system_attrs_completed_rung_0,system_attrs_completed_rung_1,system_attrs_completed_rung_2,state
0,0,0.636364,2024-09-21 13:25:56.017413,2024-09-21 13:25:56.077377,0 days 00:00:00.059964,3.578736e-07,gbtree,4.979509e-08,1.093813e-01,depthwise,6.779170e-01,1.0,NaN,NaN,NaN,NaN,0.858516,0.858516,0.858516,COMPLETE
1,1,0.636364,2024-09-21 13:25:56.080377,2024-09-21 13:25:56.222296,0 days 00:00:00.141919,3.806986e-04,gbtree,4.672493e-06,2.051228e-03,lossguide,3.156277e-01,8.0,NaN,NaN,NaN,NaN,0.970414,0.970097,0.969463,COMPLETE
2,2,0.965035,2024-09-21 13:25:56.226292,2024-09-21 13:25:56.356729,0 days 00:00:00.130437,9.713832e-02,dart,2.320976e-01,1.605471e-04,lossguide,1.947766e-07,3.0,tree,1.184663e-02,weighted,1.685327e-06,0.972844,0.975909,0.987215,COMPLETE
3,3,0.636364,2024-09-21 13:25:56.358728,2024-09-21 13:25:56.423691,0 days 00:00:00.064963,1.412132e-03,dart,6.815280e-06,1.983066e-06,lossguide,1.230868e-01,4.0,forest,1.138953e-04,weighted,8.419593e-08,NaN,NaN,NaN,COMPLETE
4,4,0.944056,2024-09-21 13:25:56.426690,2024-09-21 13:25:56.496648,0 days 00:00:00.069958,4.264679e-01,dart,5.895843e-02,6.600861e-02,lossguide,1.401019e-07,5.0,tree,1.163028e-07,uniform,2.813299e-05,NaN,NaN,NaN,COMPLETE
5,5,0.952240,2024-09-21 13:25:56.499648,2024-09-21 13:25:56.506644,0 days 00:00:00.006996,1.729343e-03,gblinear,NaN,NaN,NaN,1.178004e-06,NaN,NaN,NaN,NaN,NaN,0.952240,NaN,NaN,PRUNED
6,6,0.636364,2024-09-21 13:25:56.508642,2024-09-21 13:25:56.567608,0 days 00:00:00.058966,5.352975e-05,dart,1.543454e-07,6.670039e-02,depthwise,1.508652e-02,4.0,forest,1.811992e-02,uniform,2.656738e-03,NaN,NaN,NaN,COMPLETE
7,7,0.968935,2024-09-21 13:25:56.569608,2024-09-21 13:25:56.581601,0 days 00:00:00.011993,7.672119e-02,dart,3.719844e-04,2.289772e-03,depthwise,4.816474e-02,3.0,forest,1.764983e-02,uniform,1.446988e-06,0.968935,NaN,NaN,PRUNED
8,8,0.974324,2024-09-21 13:25:56.583600,2024-09-21 13:25:56.612583,0 days 00:00:00.028983,9.720960e-02,dart,9.899028e-06,9.103390e-06,lossguide,1.986725e-03,4.0,tree,1.225540e-06,weighted,6.224082e-06,0.975275,0.974324,NaN,PRUNED
9,9,0.973267,2024-09-21 13:25:56.614581,2024-09-21 13:25:56.640567,0 days 00:00:00.025986,2.076834e-01,gbtree,1.187692e-04,5.925305e-05,depthwise,4.746292e-06,8.0,NaN,NaN,NaN,NaN,0.973056,0.973267,NaN,PRUNED


In [8]:
# the number of brackets evaluated

study.pruner._n_brackets

5

In [9]:
# budget allocated to each initial configuration
# within each bracket

study.pruner._trial_allocation_budgets

[81, 34, 15, 8, 5]

## Unanswered questions

- How are the 50 `n_trials` specified in `study.optimize` allocated to each bracket?
- How can I identify a bracket in the result? The result being the study variable.
